In [40]:
import os
import pandas as pd 
from bs4 import BeautifulSoup
from io import StringIO


In [41]:
SCORE_DIR = 'data/scores'

In [42]:
box_scores = os.listdir(SCORE_DIR)

In [43]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith('.html')]

In [44]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select('tr.over_header')]
    [s.decompose() for s in soup.select('tr.thead')]
    return soup

In [45]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs={'id' : 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = 'team'
    cols[-1] = 'total'
    line_score.columns = cols
    
    line_score = line_score[['team', 'total']]
    return line_score

In [56]:
def read_stats(soup, team, stat):
    df = pd.read_html(StringIO(str(soup)), attrs = {'id' : f'box-{team}-game-{stat}'}, index_col= 0)[0]
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

In [93]:
def read_season_info(soup):
    nav = soup.select('#bottom_nav_container')[0]
    href = [a['href'] for a in nav.find_all('a')]
    season = os.path.basename(href[1]).split('_')[0]
    return season

In [101]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score['team'])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, 'basic')
        advanced = read_stats(soup, team, 'advanced')
        
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
        
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + '_max'
        
        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep='first'))
            base_cols = [b for b in base_cols if 'bpm' not in b]
            
        summary = summary[base_cols]
        
        summaries.append(summary)
        
    summary = pd.concat(summaries, axis=1).T  

    game = pd.concat([summary, line_score], axis=1)

    game['home'] = [0, 1]
    game_opp = game.iloc[:: -1].reset_index()
    game_opp.columns += '_opp'

    full_game = pd.concat([game, game_opp], axis=1)

    full_game['season'] = read_season_info(soup)

    full_game['date'] = os.path.basename(box_score)[:8]
    full_game['date'] = pd.to_datetime(full_game['date'], format='%Y%m%d')

    full_game['won'] = full_game['total'] > full_game['total_opp']

    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f'{len(games)} / {len(box_scores)}')

100 / 8920
200 / 8920
300 / 8920
400 / 8920
500 / 8920
600 / 8920
700 / 8920
800 / 8920
900 / 8920
1000 / 8920
1100 / 8920
1200 / 8920
1300 / 8920
1400 / 8920
1500 / 8920
1600 / 8920
1700 / 8920
1800 / 8920
1900 / 8920
2000 / 8920
2100 / 8920
2200 / 8920
2300 / 8920
2400 / 8920
2500 / 8920
2600 / 8920
2700 / 8920
2800 / 8920
2900 / 8920
3000 / 8920
3100 / 8920
3200 / 8920
3300 / 8920
3400 / 8920
3500 / 8920
3600 / 8920
3700 / 8920
3800 / 8920
3900 / 8920
4000 / 8920
4100 / 8920
4200 / 8920
4300 / 8920
4400 / 8920
4500 / 8920
4600 / 8920
4700 / 8920
4800 / 8920
4900 / 8920
5000 / 8920
5100 / 8920
5200 / 8920
5300 / 8920
5400 / 8920
5500 / 8920
5600 / 8920
5700 / 8920
5800 / 8920
5900 / 8920
6000 / 8920
6100 / 8920
6200 / 8920
6300 / 8920
6400 / 8920
6500 / 8920
6600 / 8920
6700 / 8920
6800 / 8920
6900 / 8920
7000 / 8920
7100 / 8920
7200 / 8920
7300 / 8920
7400 / 8920
7500 / 8920
7600 / 8920
7700 / 8920
7800 / 8920
7900 / 8920
8000 / 8920
8100 / 8920
8200 / 8920
8300 / 8920
8400 / 8920
8

In [102]:
games_df = pd.concat(games, ignore_index=True)

In [103]:
games_df.to_csv('nba_games.csv')